# Demonstration of SMARTpy's API

This notebook showcases an example of the use of the API of SMARTpy. It uses the input files available in the folder `examples/in/ExampleDaily`. The output files that should be generated following this tutorial are made available for comparison purposes in `examples/out/ExampleDaily`. 

Note: This tutorial is applicable to version 0.2.1 or more recent.

## Preamble

To reduce the number of paths the user needs to provide, SMARTpy requires the creation by the user of a directory (called `root` hereafter) containing two subdirectories named `in` and `out`. Within the subdirectory `in`, a subsubdirectory having the same name as the catchment to be modelled must be created: this is in this directory that SMARTpy will be looking for the input files. Following this setup, all the catchments the user desires to model can be gathered in one place, and it makes things much easier when dealing with large samples.

The structure of the root directory should follow the pattern described in the following schematic (where 'catchment' can be replaced by the actual name of the catchment):

    root/
        in/
            catchment/
                catchment.rain
                catchment.peva
                catchment.flow
                catchment.parameters
        out/

## Setting up

After having installed `smartpy` (following the instructions given in the README file), the first thing that needs to be done is to import the smartpy package to start using its modules, classes, and functions.

In [ ]:
import smartpy

### Creating an instance of the SMART class

Then, you need to create an instance of the class SMART for the catchment you desire to model. By doing so, all the input data will be collected from the input files. All arguments are mandatory except `gauged_area_m2`. If `gauged_area_m2` is provided, it means that discharge measurements are available, and SMARTpy is expecting a *.flow* file in the input folder. These measurements can be used later on in post-processing to evaluate the performance of the simulation.

In [ ]:
from datetime import datetime, timedelta

sm = smartpy.SMART(
    catchment='ExampleDaily', 
    catchment_area_m2=175.46e6, 
    start=datetime.strptime('01/01/2007 09:00:00', '%d/%m/%Y %H:%M:%S'), 
    end=datetime.strptime('31/12/2016 09:00:00', '%d/%m/%Y %H:%M:%S'), 
    time_delta_simu=timedelta(hours=1), 
    time_delta_save=timedelta(days=1), 
    warm_up_days=365, 
    in_format='csv',
    out_format='csv',
    root='examples/',
    gauged_area_m2=175.97e6
)

### Giving parameter values to the model

Now it is time to give the model its parameter values. The recommended way to go is to use a file containing the parameter values, following the template in the example file used for this tutorial. The given values will be accessible in the attribute `{SMART_instance}.parameters.values` (where SMART_instance is *sm* in this tutorial, that it is to say the name binding to the instance of SMART used).

In [ ]:
sm.parameters.set_parameters_with_file('examples/in/ExampleDaily/ExampleDaily.parameters')

Alternatively, you can avoid the creation of a *.parameters* file and directly provide them as a dictionary, as follows:

In [ ]:
sm.parameters.set_parameters_with_dict(
    {
        'T': 1.0,
        'C': 1.0,
        'H': 0.20845,
        'D': 0.24606,
        'S': 0.0001230,
        'Z': 105.26,
        'SK': 46.82,
        'FK': 315.55,
        'GK': 1066.73,
        'RK': 10.64
    }
)

### Dealing with initial conditions

If not initialised, the model states (i.e. soil moisture layers' levels, and linear reservoirs's levels) are set to zero, which will give poor streamflow predictions for the beginning of your simulation period. It is good practice to use a warm-up period to initialise the model states. This is readily available with SMARTpy, and we already defined a duration for it (i.e. 365 days) when creating our SMART instance with its `warm_up_days` attribute. If this duration is not set to zero, SMARTpy will use the first X days (here the first 365 days) of the model input time series (i.e. precipitation and potential evapotranspiration) to run the model for the given duration prior the `start` date of the simulation in order to start the actual simulation with model layers and reservoirs that are not empty (the last levels of the warm-up period are used as the assumed initial levels for the simulation period).

If one does not want to proceed this way, it is possible to set the warm-up period to 0 days. In this case, the model states are set to zero, and it is suggested that the user includes the warm-up period as part of the simulation period, and discards this period in the model outputs afterwards.

In addition to the warm-up period, SMARTpy has an additional feature to avoid starting with empty layers and reservoirs. This one is usable if the user has some general information about the hydrology in their catchment: 
* 'aar': an estimate of the annual average rainfall in millimetres,
* 'r-o_ratio': an estimate of the runoff ratio,
* 'r-o_split': an estimate of the runoff split between overland flow/drain flow/interflow/shallow groundwater flow/deep ground water flow.

If this is the case, SMARTpy will estimate the model states from this information to make an "educated guess" of the levels of the soil layers and the routing reservoirs. The information can be provided to the model as follows:

In [ ]:
sm.extra = {'aar': 1200, 'r-o_ratio': 0.45, 'r-o_split': (0.10, 0.15, 0.15, 0.30, 0.30)}

Note: this option is more advanced. If the user has access to accurate estimates for these values, they may prefer to use this option as initialisation method in place of a warm-up period, or in addition to the warm-up period.

## Simulating

The SMART instance has now all it needs to start the simulation for the catchment over the time period set up. One last instruction and the simulations will be under way. The method simulate needs to be given the parameter values, they are directly accessible from the SMART instance attributes.

In [ ]:
sm.simulate(sm.parameters.values)

## Getting the outputs

The outputs provided by SMARTpy are the simulated streamflow series and the observed streamflow series. The latter is provided if and only if the `gauged_area_m2` was provided in the SMART instance. If this is the case, the *.flow* file provided in the input folder has already been read and the data is directly available from the SMART instance (see below).

Note: in cases where `catchment_area_m2` and `gauged_area_m2` are different, the observed streamflow series provided in the the *.flow* file is rescaled accordingly assuming proportionality.

### Writing the outputs in files

If one wants to save the model outputs in files for later use. This can be done using the following instruction:

In [ ]:
sm.write_output_files(which='both')

The `which` argument can take the value 'modelled', 'observed', or 'both', to write the simulated streamflow series, the observed streamflow series, or both series, respectively. The format of the files used is the one provided in the `out_format` attribute of the SMART instance (either 'csv' or 'netcdf').

### Retrieving the outputs in numpy arrays

If one wants to directly get the model outputs in the current Python session, two getter methods of the SMART instance are available to do so:

In [ ]:
evaluation_ = sm.get_evaluation_array()
simulation_ = sm.get_simulation_array()

## Example of post-processing

The evaluation and simulation arrays returned have the same length. However, it is frequent to have days with missing discharge measurements, that would have been absent from the *.flow* file provided in the input folder. SMARTpy assigned a `nan` (not a number) value for any day with missing data, this is why the two arrays have the same length in the model outputs. However, when comparing them using an objective function, only days with available observations can be used. To deal with this, a suggested solution is as follows:

In [ ]:
import numpy as np

evaluation = evaluation_[~np.isnan(evaluation_)]
simulation = simulation_[~np.isnan(evaluation_)]

Now that the arrays have no missing value, but still the same length, they can be compared using any objective function. A suggested approach is to use the `hydroeval` Python package (https://github.com/ThibHlln/hydroeval) to get access to some of the most commonly used objective functions in hydrology:

In [ ]:
from hydroeval import *

nse_ = evaluator(nse, simulation, evaluation)  # calculates the Nash-Sutcliffe efficiency
kge_ = evaluator(kge, simulation, evaluation)  # calculates the original Kling-Gupta efficiency

Note: this example uses made-up data, this is why the efficiency of the model simulation is low, e.g. with an NSE value is of 0.39.